In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from pathlib import Path
import ot
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from utils import LoadCloudPoint, DistanceProfile

[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK
[pyKeOps] Compiling nvrtc binder for python ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK


In [6]:
import random

random.seed(10)

lcp = LoadCloudPoint(filepath="datasets/csv_files/0005_Jogging001.csv")
source_pc, target_pc = lcp.get_two_random_point_cloud()

pc = lcp.get_entire_point_cloud()

dp = DistanceProfile(source_pc, target_pc)
distance_matrix = dp.compute_L2_matrix()

Loaded point cloud data from datasets/csv_files/0005_Jogging001.csv, number of frames: 1377


In [5]:
df = pd.read_csv("datasets/csv_files/0005_Jogging001.csv")
df.head()

,X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,Z.2,X.3,...,Z.22,X.23,Y.23,Z.23,X.24,Y.24,Z.24,X.25,Y.25,Z.25
0,1623.648,2562.207,-54.364,1561.032,2728.980,-40.189,1562.895,2633.810,161.775,1184.764,...,56.574,462.429,2878.136,108.465,144.102,2791.715,297.047,80.724,2850.494,102.185
1,1627.623,2566.004,-65.609,1565.164,2732.780,-52.011,1566.692,2637.846,149.644,1189.142,...,43.926,467.083,2881.931,87.422,139.111,2799.815,268.192,81.598,2854.581,69.548
2,1631.207,2569.835,-77.109,1568.642,2736.640,-63.792,1569.912,2641.871,138.406,1193.003,...,31.467,471.141,2885.444,66.572,134.658,2807.084,238.676,82.151,2857.725,37.374
3,1634.278,2573.612,-88.228,1571.503,2740.441,-75.510,1572.542,2645.966,126.234,1196.265,...,19.195,474.584,2888.670,45.989,130.303,2813.491,209.250,83.310,2860.030,5.292
4,1636.884,2577.346,-99.153,1573.770,2744.226,-87.198,1574.548,2650.114,114.057,1198.901,...,7.133,477.313,2891.547,25.720,126.349,2819.162,180.167,84.585,2861.536,-25.269


In [7]:
pc.shape

(1377, 26, 3)

In [31]:
df = pd.DataFrame(pc.reshape(1377, 26, 3).tolist())
df = df.map(lambda x: np.nan if x[1] == 0 and x[2] == 0 else x)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,"[1623.648, 2562.207, -54.364]","[1561.032, 2728.98, -40.189]","[1562.895, 2633.81, 161.775]","[1184.764, 2692.918, 14.575]","[1133.353, 2568.54, 2.951]","[1323.044, 2417.342, 114.949]","[1028.551, 2476.519, 135.858]","[880.392, 2595.647, -50.192]","[936.44, 2618.741, -114.101]","[831.654, 2813.456, 90.135]",...,"[982.523, 2669.591, 244.278]","[995.28, 2596.972, 78.486]","[446.032, 2707.448, 188.765]","[351.912, 2598.011, 408.311]","[123.223, 2630.286, 317.281]","[42.158, 2697.756, 281.603]","[638.126, 2884.357, 56.574]","[462.429, 2878.136, 108.465]","[144.102, 2791.715, 297.047]","[80.724, 2850.494, 102.185]"
1,"[1627.623, 2566.004, -65.609]","[1565.164, 2732.78, -52.011]","[1566.692, 2637.846, 149.644]","[1189.142, 2696.818, 2.438]","[1138.007, 2572.596, -9.466]","[1327.465, 2421.934, 102.0]","[1032.494, 2481.556, 123.054]","[889.299, 2606.138, -62.345]","[943.817, 2631.873, -126.632]","[842.536, 2817.505, 77.836]",...,"[987.065, 2674.036, 231.685]","[999.884, 2598.627, 69.046]","[450.626, 2708.229, 179.86]","[357.741, 2598.177, 402.788]","[131.185, 2630.12, 315.697]","[44.251, 2697.641, 280.276]","[642.389, 2885.786, 43.926]","[467.083, 2881.931, 87.422]","[139.111, 2799.815, 268.192]","[81.598, 2854.581, 69.548]"
2,"[1631.207, 2569.835, -77.109]","[1568.642, 2736.64, -63.792]","[1569.912, 2641.871, 138.406]","[1193.003, 2700.742, -9.937]","[1142.09, 2576.564, -21.762]","[1331.393, 2426.444, 89.144]","[1036.549, 2486.377, 110.287]","[897.906, 2616.169, -74.359]","[950.76, 2644.422, -138.948]","[853.142, 2821.525, 65.458]",...,"[991.091, 2678.384, 219.134]","[1004.212, 2600.582, 59.275]","[454.69, 2709.296, 170.561]","[363.168, 2598.313, 397.094]","[139.06, 2630.156, 313.956]","[46.627, 2697.726, 279.258]","[646.174, 2887.33, 31.467]","[471.141, 2885.444, 66.572]","[134.658, 2807.084, 238.676]","[82.151, 2857.725, 37.374]"
3,"[1634.278, 2573.612, -88.228]","[1571.503, 2740.441, -75.51]","[1572.542, 2645.966, 126.234]","[1196.265, 2704.6, -22.283]","[1145.516, 2580.44, -34.074]","[1334.77, 2430.827, 76.454]","[1040.001, 2490.934, 97.447]","[905.989, 2625.693, -86.197]","[957.226, 2656.357, -151.105]","[863.325, 2825.503, 52.948]",...,"[994.522, 2682.646, 206.583]","[1007.429, 2602.897, 49.089]","[458.173, 2710.381, 160.785]","[368.16, 2598.432, 391.283]","[146.956, 2630.238, 312.185]","[49.383, 2697.259, 277.4]","[649.478, 2889.045, 19.195]","[474.584, 2888.67, 45.989]","[130.303, 2813.491, 209.25]","[83.31, 2860.03, 5.292]"
4,"[1636.884, 2577.346, -99.153]","[1573.77, 2744.226, -87.198]","[1574.548, 2650.114, 114.057]","[1198.901, 2708.348, -34.616]","[1148.238, 2584.24, -46.252]","[1337.578, 2435.116, 63.845]","[1042.884, 2495.397, 84.845]","[913.579, 2634.579, -97.858]","[963.134, 2667.661, -163.071]","[872.912, 2829.417, 40.342]",...,"[997.326, 2686.849, 194.073]","[1010.255, 2605.48, 38.578]","[461.013, 2711.781, 150.497]","[372.74, 2598.511, 385.451]","[154.867, 2630.276, 310.371]","[52.671, 2696.528, 274.142]","[652.229, 2890.928, 7.133]","[477.313, 2891.547, 25.72]","[126.349, 2819.162, 180.167]","[84.585, 2861.536, -25.269]"


In [34]:
df.isna().sum(axis=0)

0       0
1       8
2       0
3      19
4       1
5       0
6      44
7     100
8       0
9      23
10      0
11     97
12     58
13     72
14    268
15      0
16      7
17      0
18    241
19      0
20      0
21      4
22     12
23      7
24      0
25      3
dtype: int64

In [ ]:
first = lcp.get_point_cloud_at_index(161)
df1 = pd.DataFrame(first, columns=["x", "y", "z"])
df1['index'] = df1.index
fig = px.scatter_3d(df1, x="x", y="y", z="z", hover_name="index",color='index',color_continuous_scale=px.colors.qualitative.Plotly)
fig.update_layout(height=600, width=800)

fig.show()

gio: http://127.0.0.1:41489: Operation not supported
